In [1]:
import os
import re
import sys
import argparse
import logging
import subprocess
from tabulate import tabulate
from distutils.version import StrictVersion

from dbclients.tantalus import TantalusApi
from dbclients.colossus import ColossusApi
from dbclients.basicclient import NotFoundError

tantalus_api = TantalusApi()
colossus_api = ColossusApi()

JIRA_TICKET_REGEX = r".*SC-\d{4}$"

def find_latest_analysis_ticket(pool_id):
    '''
    Return the jira ticket of the newest possible analysis of the given library.
    '''
    analyses = list(colossus_api.list("analysis_information", library__pool_id = pool_id))
    latest_completion_version = None
    analysis_jira_ticket = None
    newest_version = False
    #loop over the list of analyses to find the latest possible one
    for analysis in analyses:
        status = analysis["analysis_run"]["run_status"]
        # print(status)
        current_version = analysis["version"][1:]
        # print(current_version)
        if status in ["complete", "hmmcopy_complete"]:
            if latest_completion_version:
                if StrictVersion(current_version.strip('v')) >= StrictVersion(latest_completion_version.strip('v')):
                    #print("The updated version %s" % (current_version))
                    latest_completion_version = current_version
                    analysis_jira_ticket = analysis["analysis_jira_ticket"]
                    current_analysis = analysis
            else:
                latest_completion_version = current_version
                analysis_jira_ticket = analysis["analysis_jira_ticket"]
                current_analysis = analysis
                newest_version = True
            #print(status, current_version, latest_completion_version, current_version > latest_completion_version)
        else:
            pass
            #if the latest COMPLETE analysis exisis, save the information to the dict.

    if newest_version:
        return analysis_jira_ticket

    return None



In [2]:
def get_hmmcopy_reads(jira_ticket, version, library_id):

    filenames = []
    file_resources = list(tantalus_api.list(
            "file_resource",
            filename__startswith=jira_ticket,
            filename__endswith="hmmcopy/{}_reads.csv.gz".format(library_id)
        ))

    filenames += [f["filename"] for f in file_resources]

    return filenames


In [3]:
# results_tables = scgenome.db.qc.get_qc_data(
#     snakemake.params.CNVtickets,
#     snakemake.params.local_dir,
#     sample_ids=None,
#     do_caching=False,
# )
# cn_data, segs_data, metrics_data = (
#     results_tables['hmmcopy_reads'],
#     results_tables['hmmcopy_segs'],
#     results_tables['annotation_metrics'],
# )
# metrics_data = scgenome.cnfilter.calculate_filter_metrics(
#     metrics_data,
#     cn_data,
# )

# A95626A_reads.csv.gz
# hmmcopy

In [13]:
def get_all_files(jira_ticket, version, library_id):
    filenames = []

    filenames += get_segment(jira_ticket, version, library_id)
    return filenames

def main(library_id, download_dir):
    jira_ticket = find_latest_analysis_ticket(library_id) 
    # Hoa, add condition
    if jira_ticket:
        print('Latest Jira ticket  is: {0} correspond to library id: {1}'.format(jira_ticket,library_id))
        # Define source and destination storage clients
        from_storage_name = "singlecellresults"
        to_storage_name = download_dir

        from_storage = tantalus_api.get_storage(from_storage_name)
        from_storage_client = tantalus_api.get_storage_client(from_storage_name)

        # Check if user input is a jira ticket
        # If so, check if its a analysis ticket.
        # Else check if its a library ticket and return analysis tickets

        if re.match(JIRA_TICKET_REGEX, jira_ticket):

            try:
                analysis_object = colossus_api.get("analysis_information", analysis_jira_ticket=jira_ticket)
                analysis_ticket = jira_ticket
                # library_id = analysis_object["library"]["pool_id"]
                print("{} is a valid analysis ticket\n".format(analysis_ticket))

            except NotFoundError:
                raise Exception("{} is not an analysis ticket".format(jira_ticket))

        version = analysis_object["version"]
        filenames = get_hmmcopy_reads(analysis_ticket, version, library_id)
        print(filenames)
#         filenames = get_all_files(analysis_ticket, version, library_id)
        downloaded_files = []
        for filename in filenames:
            filepath_parsed = filename.split("/")
            cell_state = filepath_parsed[1]
            analysis_type = filepath_parsed[-2]
            file = filepath_parsed[-1]
            print("cell state {0}; analysis type: {1};  files: {2}".format(cell_state,analysis_type,file))

            # REFACTOR
            # This takes care of results from old version of scpipeline
            if analysis_type == "plots":
                if "heatmap" in filename:
                    analysis_type = "hmmcopy_autoploidy"
                else:
                    analysis_type = "alignment"

            # accommadates weird cell_state_prediction naming /storage convention
            if cell_state == "cell_state_prediction":
                subdir = os.path.join(to_storage_name, library_id, cell_state)
                filepath = os.path.join(to_storage_name, library_id, cell_state, "{}_cell_state_prediction.csv".format(library_id))
            else:
                subdir = os.path.join(to_storage_name, library_id, analysis_type)
                filepath = os.path.join(to_storage_name, library_id, analysis_type, file)

            if not os.path.exists(subdir):
                os.makedirs(subdir)

            print("Downloading {} to {}".format(file, subdir))

            try:
                blob = from_storage_client.blob_service.get_blob_to_path(
                    container_name="results",
                    blob_name=filename,
                    file_path=filepath
                )
                downloaded_files.append(filepath)
            except:
                print('{} does not exist in blob.'.format(filename.split("/")[-1]))
                pass

        #clean up dir and unzip
        for root, dirs, files in os.walk(os.path.join(to_storage_name, library_id)):
            for name in files:
                if (os.path.getsize(os.path.join(root, name)) == 0 and re.search(".gz$", os.path.join(root, name))):
                    rm_cmd = ["rm", "-f", os.path.join(root, name)]
                    subprocess.check_call(rm_cmd)
                    continue

                elif re.search(".gz$", os.path.join(root, name)):
                    gunzip_cmd = ["gunzip", "-f", os.path.join(root, name)]
                    subprocess.check_call(gunzip_cmd)


        print("\n********** Download complete **********\n\n")
        for file in downloaded_files:
            print(file)
            
    else:
        print("Jira ticket do not exist for library id: {0}. Please double check the analysis".format(library_id))


In [14]:
download_dir = '/home/htran/storage/datasets/drug_resistance_DLP/SA1035'
library_ids = ['A95626A','A95646A','A95731B','A95623B','A95653B','A95646B','A95730A','A96178B',
              'A96178A','A98305A','A98290B','A98283A','A98299B','A98253A','A98256A','A98282A','A95623A']
len(library_ids)

17

In [15]:
for library_id in library_ids:
    main(library_id, download_dir)
    
# Just testing    
# library_id = 'A95626A'
# main(library_id, download_dir)

Latest Jira ticket  is: SC-3141 correspond to library id: A95626A
SC-3141 is a valid analysis ticket

['SC-3141/results/hmmcopy/A95626A_reads.csv.gz']
cell state results; analysis type: hmmcopy;  files: A95626A_reads.csv.gz

********** Download complete **********


/home/htran/storage/datasets/drug_resistance_DLP/SA1035/A95626A/hmmcopy/A95626A_reads.csv.gz
Latest Jira ticket  is: SC-2695 correspond to library id: A95646A
SC-2695 is a valid analysis ticket

['SC-2695/results/hmmcopy/A95646A_reads.csv.gz']
cell state results; analysis type: hmmcopy;  files: A95646A_reads.csv.gz

********** Download complete **********


/home/htran/storage/datasets/drug_resistance_DLP/SA1035/A95646A/hmmcopy/A95646A_reads.csv.gz
Latest Jira ticket  is: SC-3058 correspond to library id: A95731B
SC-3058 is a valid analysis ticket

['SC-3058/results/hmmcopy/A95731B_reads.csv.gz']
cell state results; analysis type: hmmcopy;  files: A95731B_reads.csv.gz

********** Download complete **********


/home/htran/s